## 1. Imports

In [1]:
import os
import io
import random
import numpy as np
from PIL import Image
import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

from tqdm import tqdm
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score
)

## 2. Config & Reproducibility

In [2]:
class CFG:
    SEED = 42
    IMG_SIZE = 224
    BATCH_SIZE = 16
    NUM_WORKERS = 0
    LR = 1e-4

    DATA_ROOT = "FFPP_CViT"
    WEIGHTS_DIR = "weights"


def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


set_seed(CFG.SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## 3. Dataset (Unified Binary Folder)

In [ ]:
class BinaryImageFolder(Dataset):
    def __init__(self, root, transform=None):
        self.samples = []
        self.transform = transform

        for label, cls in enumerate(["real", "fake"]):
            cls_dir = os.path.join(root, cls)
            for fname in os.listdir(cls_dir):
                if fname.lower().endswith((".jpg", ".jpeg", ".png")):
                    self.samples.append(
                        (os.path.join(cls_dir, fname), label)
                    )

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, torch.tensor(label, dtype=torch.float32)

## 4. JPEG Compression & Transforms

In [4]:
class JPEGCompression:
    def __init__(self, quality):
        self.quality = quality

    def __call__(self, img):
        buffer = io.BytesIO()
        img.save(buffer, format="JPEG", quality=self.quality)
        buffer.seek(0)
        return Image.open(buffer).convert("RGB")

In [5]:
train_tfms = transforms.Compose([
    transforms.Resize((CFG.IMG_SIZE, CFG.IMG_SIZE)),
    transforms.RandomAffine(2, translate=(0.02, 0.02), scale=(0.95, 1.05), shear=2),
    transforms.ColorJitter(0.6, 0.6, 0.6, 0.15),
    transforms.RandomGrayscale(p=0.2),
    transforms.RandomApply([
        transforms.GaussianBlur(3, sigma=(0.1, 2.0))
    ], p=0.3),
    transforms.RandomApply([
        transforms.RandomAdjustSharpness(0.5)
    ], p=0.3),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

eval_tfms = transforms.Compose([
    transforms.Resize((CFG.IMG_SIZE, CFG.IMG_SIZE)),
    transforms.ToTensor(),
])


def build_jpeg_tfms(q):
    return transforms.Compose([
        JPEGCompression(q),
        transforms.Resize((CFG.IMG_SIZE, CFG.IMG_SIZE)),
        transforms.ToTensor(),
    ])

## 5. Backbone

In [6]:
class CNNBackbone(nn.Module):
    def __init__(self):
        super().__init__()
        model = models.resnet34(pretrained=True)
        self.features = nn.Sequential(*list(model.children())[:-2])
        self.out_channels = 512

        for m in self.features.modules():
            if isinstance(m, nn.BatchNorm2d):
                m.eval()
                for p in m.parameters():
                    p.requires_grad = False

    def forward(self, x):
        return self.features(x)


## 6. Tokenization & Classifier

In [7]:
class EmbeddingHead(nn.Module):
    def __init__(self, in_channels, embed_dim=256, num_tokens=8):
        super().__init__()
        self.proj = nn.Conv2d(in_channels, embed_dim, 1)
        self.pool = nn.AdaptiveAvgPool2d((num_tokens, 1))

    def forward(self, x):
        x = self.proj(x)
        x = self.pool(x)
        return x.squeeze(-1).permute(0, 2, 1)

In [8]:
class TokenClassifier(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=4,
            dim_feedforward=512,
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(layer, 2)
        self.fc = nn.Linear(embed_dim, 1)

    def forward(self, x):
        x = self.encoder(x).mean(dim=1)
        return self.fc(x).squeeze(-1)

## 7. Baseline Model

In [9]:
class DeepfakeDetector(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = CNNBackbone()
        self.embedder = EmbeddingHead(self.backbone.out_channels)
        self.classifier = TokenClassifier(256)

    def forward(self, x):
        f = self.backbone(x)
        tokens = self.embedder(f)
        logits = self.classifier(tokens)
        return logits

## 8. Training Setup

In [10]:
EPOCHS = 5

model = DeepfakeDetector().to(device)
optimizer = torch.optim.Adam([
    {"params": model.backbone.parameters(),  "lr": CFG.LR * 0.2},
    {"params": model.embedder.parameters(), "lr": CFG.LR},
    {"params": model.classifier.parameters(),"lr": CFG.LR},
])

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=EPOCHS
)

criterion = nn.BCEWithLogitsLoss()

c:\Users\vk200\OneDrive\Desktop\Benchmarking\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\vk200\OneDrive\Desktop\Benchmarking\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## 9. Training & Evaluation Functions

In [11]:
def now():
    return str(datetime.datetime.now().time())[:-7]

In [12]:
def train_epoch(loader, model, optimizer):
    model.train()
    total_loss = 0.0

    for x, y in tqdm(loader, desc="Training", leave=False):
        x, y = x.to(device), y.to(device)

        logits = model(x)
        loss = criterion(logits, y)

        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)

In [13]:
@torch.no_grad()
def evaluate(loader, model, threshold=0.5):
    model.eval()
    logits, labels = [], []

    for x, y in tqdm(loader, desc="Evaluating", leave=False):
        x = x.to(device)
        logits.append(model(x).cpu())
        labels.append(y)

    logits = torch.cat(logits).numpy()
    labels = torch.cat(labels).numpy()

    probs = 1 / (1 + np.exp(-logits))
    preds = (probs >= threshold).astype(int)

    return {
        "acc": accuracy_score(labels, preds),
        "auc": roc_auc_score(labels, probs),
        "precision": precision_score(labels, preds, zero_division=0),
        "recall": recall_score(labels, preds, zero_division=0),
        "f1": f1_score(labels, preds, zero_division=0),
    }

## 10. DataLoaders

In [14]:
train_ds = BinaryImageFolder(os.path.join(CFG.DATA_ROOT, "train"), train_tfms)
val_ds   = BinaryImageFolder(os.path.join(CFG.DATA_ROOT, "val"),   eval_tfms)
test_ds  = BinaryImageFolder(os.path.join(CFG.DATA_ROOT, "test"),  eval_tfms)

train_loader = DataLoader(train_ds, CFG.BATCH_SIZE, True,  num_workers=CFG.NUM_WORKERS)
val_loader   = DataLoader(val_ds,   CFG.BATCH_SIZE, False, num_workers=CFG.NUM_WORKERS)
test_loader  = DataLoader(test_ds,  CFG.BATCH_SIZE, False, num_workers=CFG.NUM_WORKERS)

[Dataset] Loaded 38252 samples from FFPP_CViT\train
[Dataset] Loaded 13445 samples from FFPP_CViT\val
[Dataset] Loaded 13551 samples from FFPP_CViT\test


## 11. Training Loop

In [16]:
os.makedirs(CFG.WEIGHTS_DIR, exist_ok=True)

ema_auc = None
ema_decay = 0.8
best_ema_auc = -1.0
EPOCHS = 5

for epoch in range(EPOCHS):
    avg_loss = train_epoch(train_loader, model, optimizer)
    val_metrics = evaluate(val_loader, model)

    current_auc = val_metrics["auc"]

    if ema_auc is None:
        ema_auc = current_auc
    else:
        ema_auc = ema_decay * ema_auc + (1 - ema_decay) * current_auc

    print(
        f"Epoch {epoch+1:02d} | "
        f"Loss: {avg_loss:.4f} | "
        f"Val Acc: {val_metrics['acc']:.4f} | "
        f"AUC: {current_auc:.4f} | "
        f"EMA-AUC: {ema_auc:.4f} | "
        f"P: {val_metrics['precision']:.4f} | "
        f"R: {val_metrics['recall']:.4f} | "
        f"F1: {val_metrics['f1']:.4f} | "
        f"Time: {now()}"
    )

    if ema_auc > best_ema_auc:
        best_ema_auc = ema_auc
        torch.save(model.state_dict(), f"{CFG.WEIGHTS_DIR}/best_Baseline.pt")
        print(f"  ✓ Saved new best model (EMA-AUC={best_ema_auc:.4f})")

    scheduler.step()

Epoch 01 | Loss: 0.1270 | Val Acc: 0.8597 | AUC: 0.9337 | EMA-AUC: 0.9337 | P: 0.9587 | R: 0.8665 | F1: 0.9103 | Time: 08:03:24
  ✓ Saved new best model (EMA-AUC=0.9337)


Epoch 02 | Loss: 0.1196 | Val Acc: 0.8763 | AUC: 0.9255 | EMA-AUC: 0.9320 | P: 0.9342 | R: 0.9138 | F1: 0.9239 | Time: 08:14:34


Epoch 03 | Loss: 0.1011 | Val Acc: 0.8582 | AUC: 0.9309 | EMA-AUC: 0.9318 | P: 0.9587 | R: 0.8646 | F1: 0.9092 | Time: 08:26:35


Epoch 04 | Loss: 0.0842 | Val Acc: 0.8659 | AUC: 0.9352 | EMA-AUC: 0.9325 | P: 0.9598 | R: 0.8733 | F1: 0.9145 | Time: 08:39:25


Epoch 05 | Loss: 0.0604 | Val Acc: 0.8709 | AUC: 0.9366 | EMA-AUC: 0.9333 | P: 0.9566 | R: 0.8828 | F1: 0.9183 | Time: 08:51:17


## 12. FF++ Test Evaluation

In [17]:
model.load_state_dict(
    torch.load(f"{CFG.WEIGHTS_DIR}/best_Baseline.pt", map_location=device)
)
model.eval()

NUM_RUNS = 3
all_metrics = []

with torch.no_grad():
    for i in range(NUM_RUNS):
        set_seed(CFG.SEED + i)
        test_metrics = evaluate(test_loader, model)
        all_metrics.append(test_metrics)

# Average metrics
avg_metrics = {}
for key in all_metrics[0].keys():
    avg_metrics[key] = sum(m[key] for m in all_metrics) / NUM_RUNS

print("\n===== FF++ TEST (BASELINE) | AVERAGED OVER 3 RUNS =====")
for k, v in avg_metrics.items():
    print(f"{k.upper():>10}: {v:.4f}")


C:\Users\vk200\AppData\Local\Temp\ipykernel_6476\2881336639.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f"{CFG.WEIGHTS_DIR}/best_Baseline.pt", map_locatio


===== FF++ TEST (BASELINE) | AVERAGED OVER 3 RUNS =====
       ACC: 0.8667
       AUC: 0.9371
 PRECISION: 0.9667
    RECALL: 0.8678
        F1: 0.9146


## 13. JPEG Compression Robustness

In [18]:
print("\n===== JPEG ROBUSTNESS (BASELINE | 3-RUN AVG) =====")

NUM_RUNS = 3
jpeg_qualities = [100, 90, 75, 50, 30]

for q in jpeg_qualities:
    print(f"\n--- JPEG {q} ---")

    run_metrics = []

    for run_idx in range(NUM_RUNS):
        set_seed(CFG.SEED + run_idx)
        jpeg_ds = BinaryImageFolder(
            os.path.join(CFG.DATA_ROOT, "test"),
            build_jpeg_tfms(q)
        )

        jpeg_loader = DataLoader(
            jpeg_ds,
            CFG.BATCH_SIZE,
            shuffle=False,
            num_workers=CFG.NUM_WORKERS
)

        metrics = evaluate(jpeg_loader, model)
        run_metrics.append(metrics)

    avg_auc = sum(m["auc"] for m in run_metrics) / NUM_RUNS
    avg_acc = sum(m["acc"] for m in run_metrics) / NUM_RUNS
    avg_f1  = sum(m["f1"]  for m in run_metrics) / NUM_RUNS

    print(samples from
        f"AUC: {avg_auc:.4f} | "
        f"ACC: {avg_acc:.4f} | "
        f"F1: {avg_f1:.4f}"
)



===== JPEG ROBUSTNESS (BASELINE | 3-RUN AVG) =====

--- JPEG 100 ---
[Dataset] Loaded 13551 samples from FFPP_CViT\test


[Dataset] Loaded 13551 samples from FFPP_CViT\test


[Dataset] Loaded 13551 samples from FFPP_CViT\test


AUC: 0.9370 | ACC: 0.8696 | F1: 0.9169

--- JPEG 90 ---
[Dataset] Loaded 13551 samples from FFPP_CViT\test


[Dataset] Loaded 13551 samples from FFPP_CViT\test


...

[Dataset] Loaded 13551 samples from FFPP_CViT\test


AUC: 0.8909 | ACC: 0.7386 | F1: 0.8156

--- JPEG 30 ---
[Dataset] Loaded 13551 samples from FFPP_CViT\test


[Dataset] Loaded 13551 samples from FFPP_CViT\test


[Dataset] Loaded 13551 samples from FFPP_CViT\test


AUC: 0.8592 | ACC: 0.6539 | F1: 0.7397


## 14. DFDC Cross-Dataset Test

In [19]:
print("\n===== DFDC CROSS-DATASET (BASELINE | 3-RUN AVG) =====")

NUM_RUNS = 3
DFDC_ROOT = "./DFDC/validation"

dfdc_ds = BinaryImageFolder(DFDC_ROOT, eval_tfms)
dfdc_loader = DataLoader(
    dfdc_ds,
    CFG.BATCH_SIZE,
    shuffle=False,
    num_workers=CFG.NUM_WORKERS
)

all_metrics = []

for run_idx in range(NUM_RUNS):
    set_seed(CFG.SEED + run_idx)
    metrics = evaluate(dfdc_loader, model, threshold=0.5)
    all_metrics.append(metrics)

avg_metrics = {
    k: sum(m[k] for m in all_metrics) / NUM_RUNS
    for k in all_metrics[0].keys()
}

for k, v in avg_metrics.items():
    print(f"{k.upper():>10}: {v:.4f}")



===== DFDC CROSS-DATASET (BASELINE | 3-RUN AVG) =====
[Dataset] Loaded 30794 samples from ./DFDC/validation


       ACC: 0.7125
       AUC: 0.7119
 PRECISION: 0.8586
    RECALL: 0.7692
        F1: 0.8114


## 15. CelebDF Cross-Dataset Test

In [20]:
print("\n===== CELEB-DF CROSS-DATASET (BASELINE | 3-RUN AVG) =====")

NUM_RUNS = 3
CELEBDF_ROOT = "./CelebDF_images/test"

celeb_ds = BinaryImageFolder(CELEBDF_ROOT, eval_tfms)
celeb_loader = DataLoader(
    celeb_ds,
    CFG.BATCH_SIZE,
    shuffle=False,
    num_workers=CFG.NUM_WORKERS
)

all_metrics = []

for run_idx in range(NUM_RUNS):
    set_seed(CFG.SEED + run_idx)
    metrics = evaluate(celeb_loader, model, threshold=0.5)
    all_metrics.append(metrics)

avg_metrics = {
    k: sum(m[k] for m in all_metrics) / NUM_RUNS
    for k in all_metrics[0].keys()
}

for k, v in avg_metrics.items():
    print(f"{k.upper():>10}: {v:.4f}")



===== CELEB-DF CROSS-DATASET (BASELINE | 3-RUN AVG) =====
[Dataset] Loaded 8285 samples from ./CelebDF_images/test


       ACC: 0.6159
       AUC: 0.6602
 PRECISION: 0.7504
    RECALL: 0.6219
        F1: 0.6801
